In [ ]:
import torch as th
import timeit


from simplesim.risk import Risk
from simplesim.chain import Chain
from simplesim.module import Module
from simplesim.product import Product
from simplesim.csv_input import CSVInput

from simplesim.simulation.simulation import Simulation
from simplesim.simulation.risk_generator import RiskGen

In [ ]:
device = 'cuda' if th.cuda.is_available() else 'cpu'
print(device)

def test():

    num_risks = 6
    batch_size = 20
    parallel_runs = 1
    num_batches = 1000



    probability_tensor = th.rand(1, 1, num_risks) / 10
    probability_tensor.to(device)
    # countdown_tensor = th.ones_like(probability_tensor) * 10


    countdown_tensor = th.zeros(parallel_runs, num_risks).to(device)
    repair_tensor = th.randint(5, 6, countdown_tensor.shape).to(device)



    for k in range(num_batches):
        new_probability_tensor = probability_tensor.repeat(parallel_runs, batch_size, 1).to(device)
        event_tensor = th.where(new_probability_tensor > th.rand(new_probability_tensor.shape).to(device), 1, 0).to(device)
        new_event_tensor = th.zeros_like(event_tensor).to(device)

        for i in range(batch_size):

            new_event_tensor[:, i, :] = th.where((((event_tensor[:, i, :] > 0) & (countdown_tensor == 0)) | (countdown_tensor > 0)), 1, 0)
            countdown_tensor = th.where((event_tensor[:, i, :] > 0) & (countdown_tensor == 0), repair_tensor, countdown_tensor - 1)
            countdown_tensor = th.where(countdown_tensor < 0, 0, countdown_tensor)

    print(new_event_tensor)


result = timeit.timeit('test()', globals=globals(), number=1)
print(result)

In [ ]:
# Input

data_dir = CSVInput('data')
risk_assembly = data_dir.get_risk_assembly()

riskgen = RiskGen(risk_assembly)
riskgen.get_risk_vector(5)
